# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.info()
print(df.columns)
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_in

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


**Вывод**

В таблице 21525 строк.
Названия столбоц в порядке.
Тип данных


## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
# 1 В столбце days_employed значения отработанных дней со знаком минус. Вероятно ошибка при выгрузке.
# Необходимо сделать все значения положительными:
df['days_employed'] = df['days_employed'].abs()

# 2 В столбце children имеются отрицательные значения -1. Отрицательными он ибыть не могут.
# Верятно минус появился из-за ошибки при выгрузке
df['children'] = df['children'].abs()

# 3 В столбце education строки отличаются регистром. Нужно привести всё к нижнему регистру.
df['education'] = df['education'].str.lower()

# 4 В столбце days_employed присутствуют огромные значения, которые явно не являются количеством отработанных дней.
# Используя фильтр print(df[df['days_employed'] >= 20000].count()), определил что после 20 000 дней количтесво строк 3445 не изменяется.
# Это значит, что на этом реальные значеия
# закончились, а дальше начиная с сотен тысяч (приблизительно с 330 000) начнутся подозрительные. Вероятно это ошибка при выгрузке
# из-за которой значения стали на 2 порядка выше. Нужно это исправить.
df.loc[df['days_employed']>20000, 'days_employed'] = df['days_employed']/100

# 5 В столбце dob_years присутствует 101 значение 0. Эту ошибку можно удалить из-за небольшого количесвта таких строк.
df = df.loc[df['dob_years'] != 0]

# 6 В столбце 'income_type' подсчитав количество каждых категорий обнаружил, что 'безработный', 'предприниматель', 'в декрете',              
# 'студент' в сумме всего лишь 6 значений. Этого не достаточно для анализа, поэтому эти строки можно удалить:
count_income_type = df['income_type'].value_counts()
df = df.loc[df['income_type'] != 'безработный']
df = df.loc[df['income_type'] != 'предприниматель']
df = df.loc[df['income_type'] != 'в декрете']
df = df.loc[df['income_type'] != 'студент']

# 7 В столбцах 'days_employed' и 'total_income' имеются пропуски NaN одновременно в обоих столбцах. Используя bool_series = pd.isnull(df['days_employed'])
# df_nan = df[bool_series] осмотрел строки с пропусками. Никаких закономерностей появления NaN по цели кредита 'purpose' не обнаружил.
# В столбце 'income_type' пропуски отсутствуют у 'безработный', 'в декрете', 'студент'. Возможно пропуски появились вовремя выгрузки данных.
# Т.к. пропуски находятся в количественных переменных нужно заменить их медианными значениями. Среднее значение не подходит, т.к.
# немногочисленные строчки с большими значениями ссильно оттянут среднее значение в свою сторону.

# Т.к. медиана находится в упорядоченном списке без пропусков удаляем в отдельном df все пропуски:
df_not_nan = df.dropna()
# Затем находим медианы для оставшихся 4 категорий:
goss_median_days_employed = df_not_nan[df_not_nan['income_type'] == 'госслужащий']['days_employed'].sort_values().median()
komp_median_days_employed = df_not_nan[df_not_nan['income_type'] == 'компаньон']['days_employed'].sort_values().median()
pens_median_days_employed = df_not_nan[df_not_nan['income_type'] == 'пенсионер']['days_employed'].sort_values().median()
sotr_median_days_employed = df_not_nan[df_not_nan['income_type'] == 'сотрудник']['days_employed'].sort_values().median()

goss_median_total_income = df_not_nan[df_not_nan['income_type'] == 'госслужащий']['total_income'].sort_values().median()
komp_median_total_income = df_not_nan[df_not_nan['income_type'] == 'компаньон']['total_income'].sort_values().median()
pens_median_total_income = df_not_nan[df_not_nan['income_type'] == 'пенсионер']['total_income'].sort_values().median()
sotr_median_total_income = df_not_nan[df_not_nan['income_type'] == 'сотрудник']['total_income'].sort_values().median()

# Составляем дополнительный DataFrame с средними значениями:
data = [['госслужащий', 2673.404955519986, 150521.9175795368],
        ['компаньон', 1548.6375436404599, 172280.58171182536],
        ['пенсионер', 3651.763367751465, 118557.17941467627],
        ['сотрудник', 1576.0676889889892, 142594.39684740017]]
columns = ['income_type', 'days_employed', 'total_income']
df_2 = pd.DataFrame(data=data, columns=columns)
# Cоединяем основной и дополнительный DF с сохранением всех значений основного:
df = df.merge(df_2, on='income_type', how='left')
# Заменяем пропуски на соответствущие средние значения:
df['days_employed_x'] = df['days_employed_x'].fillna(df['days_employed_y'])
df['total_income_x'] = df['total_income_x'].fillna(df['total_income_y'])
# Теперь необходимо удалить вспомогательные столбцы, они больше не нужны:
df = df.drop(columns='days_employed_y')
df = df.drop(columns='total_income_y')

# 8 В столбце 'children' имеется 75 значений с количеством детей равным 20. Это ошибка, созможно вызванная опечаткой. Изменим 
# колиество детей на 2.
df['children'] = df['children'].replace(20, 2)

**Вывод**

В данных исправлены ошибки, такие как: отрицательные значения опыта работы и количества детей, разные регистры в стобце образования, огромные значения опыта работы, нулевые значения возраста и пропущенные значения в столбцах отработанных дней и общего дохода. Удалил 6 строк с редко повторяющимися категориями 'total_income'
Данные подготовлены для дальнейшего исслдеования.

### Замена типа данных

In [3]:
df['days_employed_x'] = df['days_employed_x'].astype(int)
df['total_income_x'] = df['total_income_x'].astype(int)
df

,children,days_employed_x,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income_x,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,3402,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21413,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21414,0,3439,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21415,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21416,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


**Вывод**

Теперь в таблице целочисленные значения которые легче визуально воспринимать. Использовал метод .astype() поскольку он в одно действие позволяет изменить тип данных всего столбца.

### Обработка дубликатов

In [4]:
# Для поиска дубликатов используем изученный раннее метод .duplicated(). Что бы получить число дубликатов нужно добавить sum().
df.duplicated().sum()
# Для удаления дубликатов используем метод drop_duplicates() совместно с reset_index(drop=True) для перезаписи индексов.
df.drop_duplicates().reset_index(drop=True)

,children,days_employed_x,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income_x,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,3402,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21342,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21343,0,3439,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21344,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21345,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


**Вывод**

В таблице были обнаружены 71 дубликат. Это полностью одинаковые строки. Данные строки были удалены из DF.
Дубликаты могли появиться на этапе формирования данных, записывается в базу данных. Так же дубликаты могли появится на этапе выгрузки данных из базы данных.

### Лемматизация

In [5]:
# Создадим переменную со всеми значениями целей получения кредита, но без повторов, для того, что бы познакомиться с целями кредита.
column_purpose = df['purpose'].value_counts()
print(column_purpose)

# Найдём леммы в столбце 'purpose' и запишем их в новый столбцец 'purpose_lemm'.
from pymystem3 import Mystem
m = Mystem()

def func_lemm(string):
    return m.lemmatize(string)
df['purpose_lemm'] = df['purpose'].apply(func_lemm)
df.head(5)

# Создадим новый столбец purpose_x в котором будут указаны цели кредит одним словом
def lemm(word_lemm):
    if 'жилье' in word_lemm:
        return 'недвижимость'
    if 'недвижимость' in word_lemm:
        return 'недвижимость'
    if 'автомобиль' in word_lemm:
        return 'автомобиль'
    if 'свадьба' in word_lemm:
        return 'свадьба'
    if 'образование' in word_lemm:
        return 'образование'
df['purpose_x'] = df['purpose_lemm'].apply(lemm)

# Столбцы 'purpose' и 'purpose_lemm' можно удалить
df = df.drop(columns = 'purpose')
df = df.drop(columns = 'purpose_lemm')
df.head(5)

свадьба                                   792
на проведение свадьбы                     772
сыграть свадьбу                           769
операции с недвижимостью                  673
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   650
операции с коммерческой недвижимостью     649
операции с жильем                         647
покупка жилья                             641
жилье                                     641
покупка жилья для семьи                   640
строительство собственной недвижимости    632
операции со своей недвижимостью           630
недвижимость                              630
строительство жилой недвижимости          623
покупка недвижимости                      621
строительство недвижимости                620
покупка своего жилья                      619
ремонт жилью                              609
покупка жилой недвижимости                603
на покупку своего автомобиля              502
заняться высшим образованием      

,children,days_employed_x,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income_x,purpose_x
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,3402,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба


**Вывод**

В столбце с целями кредита находятся одинаковые по смыслу, но разные по написанию цели кредита. Произведя лемматизацию по каждому стобцу, появляется возможность отследить повторяющиеся слова. Для лемматизации использовал специальную библиоткеку pymystem3. Написав функцию для нахождения лемм в строке, указываем её через метод .apply() к столбцу 'purpose'. Метод .apply() применяет функцию из аргумента к первой строке в столбце и переходит к следующей строке. Создал новый столбце, в котором указаны цели кредита одним словом по леммам.

### Категоризация данных

In [6]:
# Т.к. нужно опрделить как наличие детей, семеный статус, уровень дохода и цель кредита влияют на возврат его в срок,
# категоризацию и буду проводить для этих параметров.

# Категоризацию по количеству детей в семье проведу по значения нет дете, 1 ребёнок, 2 ребёнка 3 и более детей. 1, 2
# и 3 объединять не буду т.к. важно проследить изменения с каждым последующим ребёнком. А вот 3, 4 и 5 объеденю, т.к.
# таких значений в общей сложности всего лишь 378. Словарь создам из столбцов 'category_children' и 'category_children_id'.
def category_children(v):
    if v == 0:
        return 'нет детей'
    if v == 1:
        return '1 ребёнок'
    if v == 2:
        return '2 ребёнка'
    if v >= 3:
        return '3 и более детей'
df['category_children'] = df['children'].apply(category_children)

def category_children_id(b):
    if b == 'нет детей':
        return 0
    if b == '1 ребёнок':
        return 1
    if b == '2 ребёнка':
        return 2
    if b >= '3 и более детей':
        return 3
df['category_children_id'] = df['category_children'].apply(category_children_id)
category_children = df[['category_children', 'category_children_id']]
category_children = category_children.drop_duplicates().reset_index(drop=True)
category_children = category_children.sort_values('category_children_id')
display(category_children)

# Столбец 'family_status' тоже уже является категоризайией состоящей из 5 значений ['женат / замужем', 'гражданский брак',
# 'вдовец / вдова','в разводе', 'Не женат / не замужем']. Создам только словарь из столбцов 'family_status', 'family_status_id'.
family_status_dict = df[['family_status', 'family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
display(family_status_dict)

# По столбцу total_income_x категоризуем данные. Разобъём на следующие интервалы 50, 100, 150, 200, 400, 401+ тысяч рублей.
def category_income(x):
    if x <= 50000:
        return 'до 50тр'
    if x <= 100000:
        return '50-100тр'
    if x <= 150000:
        return '100-150тр'
    if x <= 200000:
        return '150-200тр'
    if x <= 400000:
        return '200-400тр'
    return '400+ тр'
df['category_income'] = df['total_income_x'].apply(category_income)

def category_income_id(c):
    if c == 'до 50тр':
        return 0
    if c == '50-100тр':
        return 1
    if c == '100-150тр':
        return 2
    if c == '150-200тр':
        return 3
    if c == '200-400тр':
        return 4
    if c == '400+ тр':
        return 5
df['category_income_id'] = df['category_income'].apply(category_income_id)
category_income = df[['category_income', 'category_income_id']]
category_income = category_income.drop_duplicates().reset_index(drop=True)
category_income = category_income.sort_values('category_income_id')
display(category_income)

# Категоризацию по целеи кредиты выполнил на этапе лемматизации и получил следующие категории
def purpose_id(z):
    if z == 'недвижимость':
        return 0
    if z == 'автомобиль':
        return 1
    if z == 'образование':
        return 2
    if z == 'свадьба':
        return 3
df['purpose_id'] = df['purpose_x'].apply(purpose_id)
purpose_dict = df[['purpose_x', 'purpose_id']]
purpose_dict = purpose_dict.drop_duplicates().reset_index(drop=True)
display(purpose_dict)

display(df.head(5))

,category_children,category_children_id
1,нет детей,0
0,1 ребёнок,1
3,2 ребёнка,2
2,3 и более детей,3


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


,category_income,category_income_id
5,до 50тр,0
3,50-100тр,1
1,100-150тр,2
2,150-200тр,3
0,200-400тр,4
4,400+ тр,5


,purpose_x,purpose_id
0,недвижимость,0
1,автомобиль,1
2,образование,2
3,свадьба,3


,children,days_employed_x,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income_x,purpose_x,category_children,category_children_id,category_income,category_income_id,purpose_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,1 ребёнок,1,200-400тр,4,0
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,1 ребёнок,1,100-150тр,2,1
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,нет детей,0,100-150тр,2,0
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,3 и более детей,3,200-400тр,4,2
4,0,3402,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,нет детей,0,150-200тр,3,3


**Вывод**

Произвёл категоризацию по количеству детей, семеному статусу, уровеню дохода и цели кредита. Создал словари для тех же параметров.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
children_final = df.pivot_table(index = ['category_children'], values = 'debt', aggfunc = 'mean')
children_final['%'] = children_final['debt'] * 100
display(children_final.sort_values(by='%', ascending=False))
print(df['category_children'].value_counts())

,debt,%
category_children,,
2 ребёнка,0.094991,9.499055
1 ребёнок,0.090965,9.096535
3 и более детей,0.082011,8.201058
нет детей,0.075163,7.516340


нет детей          14076
1 ребёнок           4848
2 ребёнка           2116
3 и более детей      378
Name: category_children, dtype: int64


**Вывод**

Расчитав долю единиц от всех значений для каждой категории количества детей получаем долю должников. Как видно из данных, меньше всего должников у тех, кто детей не имеет. Но с ростом количества детей, растёт доля должников.
Отличие вносит количество значений в каждой категории. Больше всего значений в категории "нет детей".
_________________________________________________________________________________________________________________________
Итак, с ростом количества детей, немного увеличивается риск невозврата кредита в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
family_status_final = df.pivot_table(index = ['family_status'], values = ['debt', 'dob_years', 'total_income_x'], aggfunc = 'mean')
family_status_final['%'] = family_status_final['debt'] * 100
display(family_status_final.sort_values(by='%', ascending=False))
display(df['family_status'].value_counts())

,debt,dob_years,total_income_x,%
family_status,,,,
Не женат / не замужем,0.097639,38.594778,166366.287196,9.763948
гражданский брак,0.092922,42.346413,164599.654068,9.292248
женат / замужем,0.075032,43.738076,166707.766223,7.503245
в разводе,0.071730,45.902110,167801.870886,7.172996
вдовец / вдова,0.064921,56.809424,142715.270157,6.492147


женат / замужем          12328
гражданский брак          4154
Не женат / не замужем     2796
в разводе                 1185
вдовец / вдова             955
Name: family_status, dtype: int64

**Вывод**

Зависимость между семейным положение и возвратом кредита в срок наблюдается. Люди без пары чаще всего имеют долги по кредитам, при этом люди живущие в гражданском браке имеют меньше долгов и люди живущие в браке ещё меньше, с относительно резким снижением значения с 0,092 до 0,075. Это говорит о том, что люди живущие в браке более серьёзно подходят к процессу взятия кредита и рассчитывают свой бюджет. Они более предсказуемы и несут обязательства перед семьёй. Поэтому долги по кредитам ставят под удар не только их но и всех членов семьи. Отсюда вытекает большая надёжность по выплате в семейных людях. В графе в разводе меньше значений чем в остальных, и возможно цифры не совсем верные. Но способность выплачивать кредит в срок связана с возрастом в этих группах. Они самые зрелые. При этом уровень дохода везде практически одинаковый.
_______________________________________________________________________________________________________________________________
Итак, люди состоящие в браке больше остальных, способны платить за кредит в срок. При этом люди в разводе, так же способны платить за кредит из-за сохранения после развода их дохода (и вероятно разделения имущества). Группа вдова/вдовец самая возрастная и хоть доходы там самые маленькие, кредит они платят исправно. Это связано с тем, что в возрасте люди больше тратят на чтото практическое чем на эмоциональное, т.е. они заплатят за кредит, а не купят айфон)

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
category_income_final = df.pivot_table(index = ['category_income'], values = ['debt'], aggfunc = 'mean')
category_income_final['%'] = category_income_final['debt'] * 100
display(category_income_final.sort_values(by='%', ascending=False))
display(df['category_income'].value_counts())

,debt,%
category_income,,
100-150тр,0.086346,8.634594
150-200тр,0.084614,8.461377
50-100тр,0.080649,8.064913
200-400тр,0.072268,7.226779
до 50тр,0.062162,6.216216
400+ тр,0.058824,5.882353


100-150тр    7192
150-200тр    4751
200-400тр    4511
50-100тр     4067
400+ тр       527
до 50тр       370
Name: category_income, dtype: int64

**Вывод**

При заработке выше 200 т.р. становится меньше долгов по кредитам. При этом значения до 50 т.р. имеют малую долю долгов из-за малого числа данных 370 шт. И на большей выборке скорее всего окажутся возле 8%.
____________________________________________________________________________________________________________________________
Итак, чем больше зароботок, тем меньше долгов за кредит. Переломное значение находится в районе 200 т.р.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
purpose_final = df.pivot_table(index = ['purpose_x'], values = ['debt', 'family_status_id'], aggfunc = 'mean')
purpose_final['%'] = purpose_final['debt'] * 100
display(purpose_final.sort_values(by='%', ascending=False))

,debt,family_status_id,%
purpose_x,,,
автомобиль,0.092964,0.987418,9.296365
образование,0.092408,0.950050,9.240759
свадьба,0.078868,1.000000,7.886841
недвижимость,0.072110,0.966540,7.211048


**Вывод**

По целям кредита очевидно, что самыми надёжными целями служат свадьба и недвижимость. А образование и автомобиль в свою очеред имеют выше долю задолжностей по кредитам. Эти данные перекликаются с данными о семейном положении и подтверждают теорию о том, что люди в браке более надёжные заёмщики

## Шаг 4. Общий вывод

В результате исследования получаем:

1) Между наличием детей и возвратом кредита в срок зависимость есть - с ростом числа детей, немного растёт доля должников.

2) Между семейным положением и возвратом кредита в срок зависимость есть - люди состоящие в браке имеют меньшую долю должников, по сравнению с теми кто не в браке или в гражданском браке. Так же низкая доля должников среди тех кто в разводе и тех кто вдовец / вдова.

3) Между уровнем дохода и возвратом кредита в срок зависимость есть - после 200 т.р. лодля должников значительно меньше.

4) Цели кредита влияют на его возврат в срок - цели недвижимость и свадьба почти равны и имеют меньшую долю должников по сравнению с целями автомобиль и образование, которые тоже почти равны между собой. 